In [1]:
import numpy as np
import cv2 as cv
import time
import os
import copy

In [2]:
path = '/home/fizzer/ros_ws/src/controller/node/'
img_folder = "pictures"
img_name = "plate_close_blue_1.png"
full_path = os.path.join(path, img_folder, img_name)

img = cv.imread(full_path,cv.IMREAD_COLOR)
hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# isolates blue
uh = 125
us = 255
uv = 255
lh = 118
ls = 40
lv = 40

lower_hsv = np.array([lh,ls,lv])
upper_hsv = np.array([uh,us,uv])

window_name = "HSV Calibrator"
cv.namedWindow(window_name)

def nothing(x):
    print("Trackbar value: " + str(x))
    pass

# create trackbars for Upper HSV
cv.createTrackbar('UpperH',window_name,0,255,nothing)
cv.setTrackbarPos('UpperH',window_name, uh)

cv.createTrackbar('UpperS',window_name,0,255,nothing)
cv.setTrackbarPos('UpperS',window_name, us)

cv.createTrackbar('UpperV',window_name,0,255,nothing)
cv.setTrackbarPos('UpperV',window_name, uv)

# create trackbars for Lower HSV
cv.createTrackbar('LowerH',window_name,0,255,nothing)
cv.setTrackbarPos('LowerH',window_name, lh)

cv.createTrackbar('LowerS',window_name,0,255,nothing)
cv.setTrackbarPos('LowerS',window_name, ls)

cv.createTrackbar('LowerV',window_name,0,255,nothing)
cv.setTrackbarPos('LowerV',window_name, lv)

font = cv.FONT_HERSHEY_SIMPLEX

while(1):
    # mask blue
    mask = cv.inRange(hsv, lower_hsv, upper_hsv)
    
    # cv.putText(mask,'Lower HSV: [' + str(lh) +',' + str(ls) + ',' + str(lv) + ']', (10,30), font, 0.5, (200,255,155), 1, cv.LINE_AA)
    # cv.putText(mask,'Upper HSV: [' + str(uh) +',' + str(us) + ',' + str(uv) + ']', (10,60), font, 0.5, (200,255,155), 1, cv.LINE_AA)

    cv.imshow(window_name,mask)

    k = cv.waitKey(1) & 0xFF
    if k == 27:
        break
    # get current positions of Upper HSV trackbars
    uh = cv.getTrackbarPos('UpperH',window_name)
    us = cv.getTrackbarPos('UpperS',window_name)
    uv = cv.getTrackbarPos('UpperV',window_name)
    upper_blue = np.array([uh,us,uv])
    # get current positions of Lower HSCV trackbars
    lh = cv.getTrackbarPos('LowerH',window_name)
    ls = cv.getTrackbarPos('LowerS',window_name)
    lv = cv.getTrackbarPos('LowerV',window_name)

    upper_hsv = np.array([uh,us,uv])
    lower_hsv = np.array([lh,ls,lv])

    time.sleep(.1)

cv.destroyAllWindows()

Trackbar value: 125
Trackbar value: 255
Trackbar value: 255
Trackbar value: 118
Trackbar value: 40
Trackbar value: 40
Trackbar value: 57
Trackbar value: 59
Trackbar value: 63
Trackbar value: 65
Trackbar value: 66
Trackbar value: 71
Trackbar value: 80
Trackbar value: 86
Trackbar value: 96
Trackbar value: 97
Trackbar value: 96
Trackbar value: 95
Trackbar value: 94
Trackbar value: 91
Trackbar value: 88
Trackbar value: 90
Trackbar value: 91
Trackbar value: 92
Trackbar value: 93
Trackbar value: 96
Trackbar value: 97
Trackbar value: 99
Trackbar value: 102


KeyboardInterrupt: 